# json_normalize()
* In this video, we will talk about how to use the pandas json_normalize function and go in depth on the arguments you can pass. 
* Within the video description, I have included a link to the github repository that has this jupyter notebook and a link to the dataset I'm using in this video. 

In [ ]:
import pandas as pd
import json

# Opening JSON Files

In [ ]:
# open json as a text file 
f = open('sea_level_rise.json','r')

# convert to python dictionary 
file = json.loads(f.read())
print(type(file))
# file

# Navigating JSON Files

* When I say navigating the JSON file what we are actually doing is navigating a python dictionary version of the json file.
* Dictionaries in python, just like json are based on a key and value pair logic. 
* Central to naviagating any JSON file, therefore is knowing what all of the keys are. 

In [ ]:
# First Level 
file.keys()
# Many JSON data files are initally divided into two main parts 

In [ ]:
# Second Level
file['meta'].keys()

In [ ]:
# Third Level
file['meta']['view'].keys()
# Eventually you will get to the point where you dont have keys anymore. you will just have values or lists of values (arrays). 

# Selecting Values

In [ ]:
file['meta']['view']['name']

# Default Arguments
* The json_normalize function attempts to create an easy to read pandas dataframe. This is an extremely useful tool and there are multiple arguments you can use to produce a really clean output. 
* Let's first use an example with no arguments.

In [ ]:
data = pd.json_normalize(file)
data

# record_path argument
* Now we will use the record path argument to step into this data a little bit more. 
* record_path can take either a string or list of strings. Accpetable strings are keys within the json file.
* When passing a single key a data frame will be generated from that level. 
* When passing a list of keys you are specifying the exact level and location you want the dataframe to be generated from. 

In [ ]:
data = pd.json_normalize(file,record_path='data')
data
# You can see that this dataframe does not have any column headers, but those can be found in the meta data.

## Generating Column List
* In order to generate the column header list for the dataframe above, we will use the json_normalize function again.
* This time we will be using an extended record path by passing in a list of strings rather than a single string.
* All column names are found in the meta data so we will make the walk into it.

In [ ]:
cols = pd.json_normalize(file,record_path=['meta','view','columns'])
cols

In [ ]:
# Looking at the dataframe, the "name" column apprears to contain the column names we need for our main dataframe. 
# We will grab the entire column and convert it to a dictionary. 
col_list = cols['name'].to_dict()
col_list

In [ ]:
# Now that we have the column names, we can change the column headers in the dataframe with the rename function. 
# Unfortunately, unlike read_csv there is no way to pass the column names within the json_normalize function. So you must do it after the fact. 
data.rename(columns=col_list,inplace=True)
data

# meta argument
* We can add additional rows to our dataframe using the meta data section and meta argument. 

In [ ]:
data = pd.json_normalize(file,record_path='data',meta='meta')
data

# meta_prefix argument
* Up to this point we have used a json file where there were two main keys on the first level, but not all json files will be like that. Rather than having two main keys like "data" and "meta", you may have multiple keys on the first level creating what is known as a flatter json structure. 
* Assuming you want to include meta data, you can use the meta_prefix argument to help you organize the additional columns.
* We are going to use a modified version of the sea_level_rise.json file. I removed the meta and view keys so that all meta keys are on the same level as the data key. 


In [ ]:
import pandas as pd
import json

In [ ]:
# open json as a text file 
f2 = open('sea_level_rise_flat.json','r')

# convert to python dictionary 
file2 = json.loads(f2.read())
print(type(file2))
# file2

# Navigating the Modified File
* Now that we have this "flatter" version in a dictionary object, lets explore it like we did the first using the keys() function. 

In [ ]:
file2.keys()
# Notice in this adjusted files that there are more keys at the inital level when compared with the other file

In [ ]:
data = pd.json_normalize(file2,record_path='data',meta=['name','category','metadata'],meta_prefix='meta.')
data

# record_prefix argument
* In addition to adding a meta prefix, you can add record prefixes too.  

In [ ]:
data = pd.json_normalize(file2,record_path='data',meta=['name','category'],meta_prefix='meta.',record_prefix='data.')
data

# errors argument
* The default behavior for the errors argument is to raise errors and exit the execution.
* This occurs when meta key valuesare missing. 
* To avoid this, you can set the error argument to "ignore". 
* When you do this np.nan values are placed in the missing spots and the dataframe can still be created successfully. 

In [ ]:
data = pd.json_normalize(file2,record_path='data',meta=['name','category','nonExistentMeta'],meta_prefix='meta.',record_prefix='data.')
data

In [ ]:
data = pd.json_normalize(file2,record_path='data',meta=['name','category','nonExistentMeta'],meta_prefix='meta.',record_prefix='data.',errors='ignore')
data

# sep argument
* Going back to the original file, we will now demonstrate how to use the sep argument.
* sep allows you to change the path separator.
* While this may look similar to prefixes, there is a fundamental difference. 
* Prefixes overwrite the JSON defined paths and are used for organizing "flatter" json files. Separators are JSON defined paths used to show where the data is coming from in "hierarchical" json files. 
* Remember back to the first example we did with no arguments. 

In [ ]:
data = pd.json_normalize(file)
data

In [ ]:
data = pd.json_normalize(file,sep='>>>')
data

# max_level argument
* By default json normalize goes through all levels of the json file. 
* But you can specify how far you would like it to go. 

In [ ]:
data = pd.json_normalize(file,max_level=0)
data

In [ ]:
data = pd.json_normalize(file,max_level=2)
data

In [ ]:
data = pd.json_normalize(file,max_level=3)
data
#NOTE: the deeper you go, the more columns you will get. 

# Alternate way to get column headers

In [ ]:
file['meta']['view']['columns']

In [ ]:
col_list = []
for name in file['meta']['view']['columns']:
    col_list.append(name['name'])

In [ ]:
col_list